In [1]:
# ============================================================
# Notebook setup: run this before everything
# ============================================================
# -- Copied from lecture
%load_ext autoreload
%config IPCompleter.greedy=True
%autoreload 1
%aimport util
import logging

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.neighbors import KernelDensity
from sklearn.preprocessing import StandardScaler

from util import util

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Control figure size
interactive_figures = False
if interactive_figures:
    # Normal behavior
    %matplotlib widget
    figsize=(9, 3)
else:
    # PDF export behavior
    figsize=(14, 4)

raw_data = util.load_dataset('7_gecco2019_train_water_quality.csv')
raw_data = util.impute_missing_values(raw_data)

# Multivariate Kernel Density Estimation
The first approach presented in the lecture is **Kernel Density Estimation**

In order to employ **KDE**, we need to determine the optimal **Kernel Function** and **Bandwidth**.
Since we have multiple columns, we cannot use the Rule Of Thumb for the latter. Therefore, we need to optimize the following term according to the lecture:
$$
\mathop{\arg\max}_{h} \mathbb{E}_{x \sim f(x), \bar{x} \sim f(x)}\left[ L(h, x, \bar{x})\right]
$$
where
- $$
L(h, x, \bar{x}) = \prod_{i=1}^m \hat{f}(x_i, \bar{x}_i, h)
$$
- $\hat{f}$ is the density estimator (which outputs a probability)
- $\bar{x}$ the training set

according to the lecture.

## Preprocessing
KDE is a non-parametric method, meaning that it does not make any assumptions about the shape and the distribution of the data. Therefore, we need to preprocess the data before applying KDE. First, we have to make sure that our data is free from missing values. As seen before, a linear interpolation approach yields the best results. Therefore, we will interpolate missing values using this method. Then, we have to apply a sliding window approach using the `aggregation_length` parameter explained before and aggregate the data into windows. This makes sure that we capture temporal correlations between data points and additionally removes noise in the data. The final step of our preprocessing pipeline is to standardize the data.

In [2]:
# Preprocess the data (interpolating missing values and applying sliding window)
kde_data_df = util.impute_missing_values(raw_data)
kde_data_df = util.apply_sliding_window_and_aggregate(kde_data_df)

# Identify the features to be used for KDE
kde_features = util.get_feature_columns(kde_data_df)

# Standardize the data (KDE assumes normally distributed data)
kde_scaler = StandardScaler()
kde_data_df[kde_features] = kde_scaler.fit_transform(kde_data_df[kde_features])

print(kde_data_df.head())

                     window_0  window_1  window_2  window_3  window_4  \
Time                                                                    
2017-07-01 00:09:00 -1.231517  1.060557 -0.382443 -0.408179 -1.684389   
2017-07-01 00:10:00 -1.242696  1.034397 -0.350244 -0.192011 -1.673169   
2017-07-01 00:11:00 -1.231517  0.982218 -0.342876 -0.315704 -1.696363   
2017-07-01 00:12:00 -1.231517  0.982218 -0.333053 -0.365718 -1.681567   
2017-07-01 00:13:00 -1.231517  1.008378 -0.328141 -0.076374 -1.697380   

                     window_5  window_6  window_7  window_8  window_9  ...  \
Time                                                                   ...   
2017-07-01 00:09:00 -2.181417 -1.242701  1.034405 -0.350237 -0.192011  ...   
2017-07-01 00:10:00 -2.209378 -1.231522  0.982226 -0.342870 -0.315704  ...   
2017-07-01 00:11:00 -2.195122 -1.231522  0.982226 -0.333047 -0.365719  ...   
2017-07-01 00:12:00 -2.215175 -1.231522  1.008386 -0.328135 -0.076374  ...   
2017-07-01 00:13:00 

## Bandwidth Choice in Multivariate KDE
The bandwidth parameter $h$ is a hyperparameter that controls the smoothness of the estimated density. It has to be chosen such that it maximizes the likelihood of the data:

$$
argmax_h \mathbb{E}_{x \sim f(x), \bar{x} \sim f(x)}\left[ L(h, x, \bar{x})\right],
$$

where $f(x)$ is the true distribution function and $L(h, x, \bar{x)$ is the log-likelihood function.

 In the lecture, we discussed the following methods to determine the optimal bandwidth:
- The rule of thumb method: This method uses a rule of thumb to estimate the optimal bandwidth based on the data. It involves calculating the standard deviation of the data and using it as the bandwidth. However, this method is not always reliable, as it can be sensitive to the scale of the data.
- Cross-validation: This method involves splitting the data into training and validation sets and using the validation set to estimate the optimal bandwidth. The model is then trained on the training set and evaluated on the validation set. This method is more robust than the rule of thumb method, as it can handle non-normal data and avoid overfitting.
- Grid search: This method involves searching over a grid of possible bandwidths and selecting the one that gives the best performance. This method is computationally expensive, but it can be useful when the data is not normally distributed.

In our case, we will use a grid search to determine the optimal bandwidth. We will use the F1-score to evaluate the performance of the KDE model using a given bandwidth. The grid search will be performed using the `sklearn.model_selection.GridSearchCV` class. As we're working with time-series data, we will use a `sklearn.model_selection.TimeSeriesSplit` cross-validation strategy to split the data into training and validation sets.

The downside of the grid search strategy is that it is computationally very expensive, especially for large datasets, as it runs a cross-validation for each possible bandwidth value.

In [3]:
# Select only features columns.
X = kde_data_df[kde_features]

# Define a reasonable range of bandwidths
bandwidths = np.linspace(0.7, 0.8, 5)

# Create a TimeSeriesSplit instance to ensure training data precedes test data
tscv = TimeSeriesSplit(n_splits=5)

# Set up GridSearchCV with verbose output to monitor progress; n_jobs=-1 uses all cores
grid = GridSearchCV(KernelDensity(kernel='gaussian'), {'bandwidth': bandwidths}, cv=tscv, verbose=3, n_jobs=-1)

# Run the grid search to fit the KDE model and determine the best bandwidth
grid.fit(X)

best_bw = grid.best_params_['bandwidth']
print("Best bandwidth:", grid.best_params_['bandwidth'])
print("Best log-likelihood score:", grid.best_score_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 4/5] END ............bandwidth=0.725;, score=-1221040.668 total time= 8.0min
[CV 1/5] END ..............bandwidth=0.8;, score=-1400469.650 total time= 2.6min
Best bandwidth: 0.7
Best log-likelihood score: -1827092.5103767694


## Train the KDE
Next, we train the KDE with the optimal bandwidth. Additionally, we compute the log likelihood scores for each data point. This score is a measure of how likely a data point is to be generated by the underlying distribution. Higher scores indicate a higher likelihood of being generated by a Gaussian distribution.

Due to the high dimensionality of the data, training and evaluating the KDE model suffer from the "curse of dimensionality". This means that the model becomes increasingly difficult to train and evaluate as the number of dimensions increases.

In [4]:
# Select only features columns.
X = kde_data_df[kde_features]

# Train KDE with best bandwidth.
final_kde = KernelDensity(kernel='gaussian', bandwidth=best_bw)
final_kde.fit(X)

# Compute likelihood scores for the training data.
log_likelihood = final_kde.score_samples(X)

[CV 1/5] END ............bandwidth=0.725;, score=-1292972.402 total time= 2.4min
[CV 2/5] END .............bandwidth=0.75;, score=-2182875.470 total time= 4.4min
[CV 3/5] END ............bandwidth=0.775;, score=-2675668.799 total time= 5.0min
[CV 2/5] END ............bandwidth=0.725;, score=-2204778.200 total time= 4.4min
[CV 4/5] END .............bandwidth=0.75;, score=-1265856.338 total time= 8.2min
[CV 5/5] END ..............bandwidth=0.7;, score=-1623409.327 total time= 9.1min
[CV 2/5] END ..............bandwidth=0.8;, score=-2153508.216 total time= 3.8min
[CV 3/5] END ..............bandwidth=0.7;, score=-2852069.041 total time= 4.6min
[CV 5/5] END .............bandwidth=0.75;, score=-1664421.965 total time= 9.0min
[CV 2/5] END ..............bandwidth=0.7;, score=-2227191.085 total time= 4.3min
[CV 3/5] END .............bandwidth=0.75;, score=-2726704.334 total time= 4.8min
[CV 3/5] END ..............bandwidth=0.8;, score=-2639169.914 total time= 4.6min
[CV 3/5] END ............ban

## Threshold Optimization
Now, we need to define a threshold to separate normal data from anomalous data. We will use a simple threshold optimization approach using the validation set. First, we define the percentiles to test. Then, we compute precision, recall, and F1-score for each percentile. These are the preferred metrics when working with big class imbalances. Finally, we select the percentile with the highest F1-score.

In [5]:
# Load validation data
val_data = util.load_dataset('8_gecco2019_valid_water_quality.csv')

# Preprocess validation data
kde_val_data_df = util.impute_missing_values(val_data)
kde_val_data_df = util.apply_sliding_window_and_aggregate(kde_val_data_df)

# Standardize the data (KDE assumes normally distributed data)
kde_val_data_df[kde_features] = kde_scaler.transform(kde_val_data_df[kde_features])

# Compute scores for validation data
log_likelihood_val = final_kde.score_samples(kde_val_data_df[kde_features])

# Define percentiles to test.
percentiles = np.arange(0.1, 2.1, 0.1)

# For storing the results.
results = []

for p in percentiles:
    # Get predictions and threshold
    y_pred, threshold = util.get_predictions_from_log_likelihood(log_likelihood_val, p)

    # Compute performance
    f1, precision, recall = util.compute_model_performance(y_pred, kde_val_data_df['Event'])

    # Store results.
    results.append((p, threshold, precision, recall, f1))

# Convert to DataFrame for better visualization.
df_results = pd.DataFrame(results, columns=['Percentile', 'Threshold', 'Precision', 'Recall', 'F1-score'])

# Display results.
kde_best_percentile = df_results.loc[df_results['F1-score'].idxmax()]
print(df_results)
print(f"Best KDE model with percentile {kde_best_percentile['Percentile']} and threshold {kde_best_percentile['Threshold']} achieves an F1-score of {kde_best_percentile['F1-score']}")

    Percentile     Threshold  Precision    Recall  F1-score
0          0.1 -75948.705540   0.181818  0.030211  0.051813
1          0.2 -71218.840030   0.100000  0.033233  0.049887
2          0.3 -66921.981164   0.084848  0.042296  0.056452
3          0.4 -62831.763739   0.082192  0.054381  0.065455
4          0.5 -58837.337355   0.065693  0.054381  0.059504
5          0.6 -55024.673594   0.057751  0.057402  0.057576
6          0.7 -51718.769490   0.049479  0.057402  0.053147
7          0.8 -48605.275273   0.045662  0.060423  0.052016
8          0.9 -45486.712633   0.040568  0.060423  0.048544
9          1.0 -42791.386100   0.038321  0.063444  0.047782
10         1.1 -40338.049568   0.041459  0.075529  0.053533
11         1.2 -38085.564301   0.041096  0.081571  0.054656
12         1.3 -36084.714872   0.039326  0.084592  0.053691
13         1.4 -34365.271081   0.039113  0.090634  0.054645
14         1.5 -32837.884047   0.038929  0.096677  0.055507
15         1.6 -31527.391661   0.036530 

## Compute number of anomalies
Now, we compute the number of anomalies for the identified threshold.

In [6]:
# Compute number of anomalies
anomalies, threshold = util.get_predictions_from_log_likelihood(log_likelihood, kde_best_percentile['Percentile'])

print(f"Anomaly threshold: {threshold:.2f}")
print(f"Number of anomalies: {np.sum(anomalies)}")

# Add anomaly labels to DataFrame
kde_data_df['Anomaly_Score'] = log_likelihood
kde_data_df['Anomaly'] = anomalies

print(kde_data_df['Anomaly_Score'].head())

Anomaly threshold: -53.46
Number of anomalies: 2517
Time
2017-07-01 00:09:00   -48.804299
2017-07-01 00:10:00   -48.801081
2017-07-01 00:11:00   -48.808160
2017-07-01 00:12:00   -48.840240
2017-07-01 00:13:00   -48.888673
Name: Anomaly_Score, dtype: float64


## Confusion Matrix and Classification Report
To measure the performance of our anomaly detection model on the training data, we can use the confusion matrix and classification report.

In [7]:
# Convert Boolean to integers for evaluation
y_true = kde_data_df['Event'].astype(int)  # Actual contamination events
y_pred = kde_data_df['Anomaly'].astype(int)  # Detected anomalies

# Print performance metrics
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("Classification Report:\n", classification_report(y_true, y_pred))

Confusion Matrix:
 [[129879   2263]
 [    75    254]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99    132142
           1       0.10      0.77      0.18       329

    accuracy                           0.98    132471
   macro avg       0.55      0.88      0.58    132471
weighted avg       1.00      0.98      0.99    132471



The model achieves a very high overall accuracy of 98%, driven primarily by its excellent performance on the dominant class (class 0), where it correctly identifies almost all cases (98% recall, nearly 100% precision). However, this high accuracy masks a significant weakness in detecting the minority class (class 1). For class 1, although the recall is moderately high at 77% (meaning that most actual positives are captured), the precision is extremely low at just 10%, indicating that a large number of false positives are being flagged. This imbalance suggests that while the KDE model is very effective at recognizing the majority class, it struggles to reliably identify the minority class, due to the imbalanced nature of the training data.

## Performance on Test Data
Finally, we evaluate the model's performance on the test data. This is an important step that allows us to assess how well the model generalizes to unseen data. First, we have to perform the same preprocessing steps as with the training data. In the last step, we can apply the KDE to the test data and compute the confusion matrix and classification report.

In [8]:
# Load test data
test_data = util.load_dataset('6_gecco2019_test_water_quality.csv')

# Preprocess test data
kde_test_data_df = util.impute_missing_values(test_data)
kde_test_data_df = util.apply_sliding_window_and_aggregate(kde_test_data_df)

# Standardize the data (KDE assumes normally distributed data)
kde_test_data_df[kde_features] = kde_scaler.transform(kde_test_data_df[kde_features])

# Compute scores for test data
X_test = kde_test_data_df[kde_features]
log_likelihood_test = final_kde.score_samples(X_test)  # Higher is more normal, lower is more anomalous

# Get predictions and threshold
anomalies_test, threshold_test = util.get_predictions_from_log_likelihood(log_likelihood_test, kde_best_percentile['Percentile'])

# Add anomaly labels to DataFrame
kde_test_data_df['Anomaly_Score'] = log_likelihood_test
kde_test_data_df['Anomaly'] = anomalies_test

# Convert Boolean to integers for evaluation
y_true = kde_test_data_df['Event'].astype(int)  # Actual contamination events
y_pred = kde_test_data_df['Anomaly'].astype(int)  # Detected anomalies

# Print performance metrics
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("Classification Report:\n", classification_report(y_true, y_pred))

Confusion Matrix:
 [[30867   475]
 [  172   127]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99     31342
           1       0.21      0.42      0.28       299

    accuracy                           0.98     31641
   macro avg       0.60      0.70      0.64     31641
weighted avg       0.99      0.98      0.98     31641



On the test data, the model maintains a high overall accuracy of 98% and continues to perform well for the majority class (class 0) with an accuracy of 99% and a recall of 98%. However, compared to the training results, the performance for the minority class (Class 1) has changed: while the recall in training was higher at 77%, the recall in testing drops to 42%, indicating that the model is now missing more true positives. Conversely, the accuracy for Class 1 improves from 10% in the training set to 21% in the test set, meaning that the model is more likely to correctly predict Class 1 with the unseen data. Although the F1 score has improved, it is still very low at 0.28. These differences indicate a possible problem of overfitting or that the model's ability to generalize to the minority class is limited, as its sensitivity to the test data has decreased despite a slight improvement in prediction accuracy for the minority class.